In [ ]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score
import tensorflow as tf
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# Reset TensorFlow graph
tf.compat.v1.reset_default_graph()

# Paths and data
checkpoint_path = "E:/my_models/750_if_new_best_model.keras"
save_path = "E:/my_plots/750_if_new_prediction_plots/"
mutated_data = np.load("/content/MUTATION_DATA_TRAINING_6000.npz", allow_pickle=True, mmap_mode='r')
nonmutated_data = np.load("/content/AUGMENTED_DATA_TRAINING_6000.npz", allow_pickle=True, mmap_mode='r')
csv_file = "cry1realvariations (1).csv"  # Assuming this CSV contains mutation data for anomaly detection

# Anomaly detection using Isolation Forest
def load_and_get_anomaly_scores(csv_file):
    df = pd.read_csv(csv_file)
    df = df[df["variation_type"].str.contains("intron_variant", na=False, case=False)]
    # Assuming the CSV file has columns 'Mutation ID' and 'Allele Frequency'
    data_id_array = df['_displayName'].values[:6000].reshape(-1, 1)  # Get the first 1000 Mutation IDs
    array_data = df['AF'].values[:6000].reshape(-1, 1)  # Get the first 1000 Allele Frequencies

    # Initialize and fit the Isolation Forest model
    clf = IsolationForest(contamination=0.01, random_state=42)
    clf.fit(array_data)

    # Get anomaly scores and predictions
    anomaly_scores = clf.decision_function(array_data)  # Negative scores represent outliers (anomalous)
    predictions = clf.predict(array_data)  # -1 for anomaly, 1 for normal

    # Invert the anomaly scores (as lower scores indicate anomalies, we invert so higher scores are more "normal")
    inverted_anomaly_scores = -anomaly_scores

    # Filter out NaN values and prepare the data for plotting
    valid_indices = ~np.isnan(array_data)
    valid_indices = valid_indices.flatten()
    data_id_array = data_id_array[valid_indices]
    array_data = array_data[valid_indices]
    inverted_anomaly_scores = inverted_anomaly_scores[valid_indices]
    predictions = predictions[valid_indices]

    return inverted_anomaly_scores, data_id_array, array_data, predictions

# Get anomaly scores
anomaly_scores, _, _, _ = load_and_get_anomaly_scores(csv_file)

# Function to load sequences and their labels
def load_sequences(data, label):
    encoded_sequences = None
    input_shape = None

    for key in data.files:
        temp_sequences = data[key]
        print(f"Checking key '{key}': shape {temp_sequences.shape}")  # Debugging

        if temp_sequences.ndim == 2:  # If 2D, reshape to 3D for LSTM
            temp_sequences = np.expand_dims(temp_sequences, axis=1)  # (samples, 1, features)

        if temp_sequences.ndim == 3:
            encoded_sequences = temp_sequences
            input_shape = (encoded_sequences.shape[1], encoded_sequences.shape[2])
            print(f"Using key '{key}' with reshaped shape {encoded_sequences.shape}")
            break
        else:
            print(f"Skipping '{key}': Unexpected shape {temp_sequences.shape}")

    if encoded_sequences is None:
        raise ValueError(f"No valid encoded sequences found in {label} file.")

    return encoded_sequences, input_shape

# Load data
mutated_sequences, input_shape = load_sequences(mutated_data, "mutated")
nonmutated_sequences, _ = load_sequences(nonmutated_data, "nonmutated")

mutated_labels = np.ones(mutated_sequences.shape[0])  # 1 for mutated
nonmutated_labels = np.zeros(nonmutated_sequences.shape[0])  # 0 for non-mutated

# Step 1: Generate random Mutation IDs for non-mutated data
nonmutated_data_with_ids = np.array([f"NonMut_{i}" for i in range(len(nonmutated_data.files))])

# Step 2: Add noise to the Allele Frequency for the non-mutated data
allele_frequency_mutated = pd.read_csv(csv_file)['AF'].values[:6000]  # Allele Frequency of mutated data

# Generate random noise (Gaussian noise with mean 0 and std deviation SAME as non-mutated data)
nonmutated_allele_frequency = np.random.normal(np.mean(allele_frequency_mutated), np.std(allele_frequency_mutated), len(nonmutated_data.files))

# Step 3: Incorporate noise into non-mutated data for Isolation Forest
df_nonmutated_with_noise = pd.DataFrame({
    '_displayName': nonmutated_data_with_ids,
    'AF': nonmutated_allele_frequency
})

# Step 4: Apply Isolation Forest on non-mutated data
clf = IsolationForest(contamination=0.01, random_state=42)
clf.fit(df_nonmutated_with_noise['AF'].values.reshape(-1, 1))

# Get anomaly scores and predictions for non-mutated data
nonmutated_anomaly_scores = clf.decision_function(df_nonmutated_with_noise['AF'].values.reshape(-1, 1))
nonmutated_predictions = clf.predict(df_nonmutated_with_noise['AF'].values.reshape(-1, 1))

# Invert the anomaly scores (lower scores = anomalies)
nonmutated_inverted_anomaly_scores = -nonmutated_anomaly_scores

# Step 5: Append anomaly scores to non-mutated sequences
nonmutated_sequences_with_anomalies = nonmutated_sequences.copy()  # Make a copy of non-mutated sequences

# Reshape anomaly scores to match the sequence shape
nonmutated_anomaly_scores_reshaped = nonmutated_inverted_anomaly_scores.reshape(-1, 1)  # Shape (6000, 1)

# Expand the anomaly scores to 3D (samples, timesteps, features) for compatibility with sequence shape
nonmutated_anomaly_scores_reshaped = np.expand_dims(nonmutated_anomaly_scores_reshaped, axis=-1)  # Shape (6000, 1, 1)

# Update the last feature dimension of non-mutated sequences with the anomaly score
nonmutated_sequences_with_anomalies[:, :, -1:] = nonmutated_anomaly_scores_reshaped  # Add anomaly score to non-mutated sequences

# Step 6: Apply Isolation Forest on mutated data
mutated_sequences_with_anomalies = mutated_sequences.copy()  # Make a copy of mutated_sequences

# Assuming you already have anomaly scores for mutated data (from previous steps), append them:
anomaly_scores_reshaped = anomaly_scores[:mutated_sequences.shape[0]].reshape(-1, 1)  # Shape (1000, 1)

# Expand anomaly_scores_reshaped to 3D (samples, timesteps, features) for mutated data
anomaly_scores_reshaped = np.expand_dims(anomaly_scores_reshaped, axis=-1)  # Shape (1000, 1, 1)

# Update the last feature dimension of mutated sequences with the anomaly score
mutated_sequences_with_anomalies[:, :, -1:] = anomaly_scores_reshaped  # Add anomaly score to mutated sequences

# Step 7: Concatenate mutated and non-mutated sequences with anomalies
X_with_anomalies = np.concatenate([mutated_sequences_with_anomalies, nonmutated_sequences_with_anomalies], axis=0)

# Step 8: Concatenate labels (mutated labels and non-mutated labels)
y_mutated = np.ones(mutated_sequences.shape[0])  # Labels for mutated sequences (1)
y_nonmutated = np.zeros(nonmutated_sequences.shape[0])  # Labels for non-mutated sequences (0)

# Concatenate the mutated and non-mutated labels
y = np.concatenate([y_mutated, y_nonmutated], axis=0)

Checking key 'arr_0': shape (6000, 410000)
Using key 'arr_0' with reshaped shape (6000, 1, 410000)
Checking key 'arr_0': shape (6000, 410000)
Using key 'arr_0' with reshaped shape (6000, 1, 410000)


In [ ]:
X_with_anomalies, y = shuffle(X_with_anomalies, y, random_state=42)

# Split the dataset into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_with_anomalies, y, test_size=0.1, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1 / 0.9, random_state=42, stratify=y_train_val
)

# Now we have 80% training, 10% validation, and 10% test

# RNN Model
def rnn_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True, activation="relu"),
        Dropout(0.5),  # First dropout layer

        LSTM(32, activation="relu", return_sequences=False),
        Dropout(0.5),  # Second dropout layer

        Dense(16, activation="relu"),
        Dropout(0.5),  # Third dropout layer

        Dense(1, activation="sigmoid")  # Output layer
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Train the model
model = rnn_model(input_shape)
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True, verbose=1)
csv_logger = CSVLogger("training_log.csv")

history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val),
                    callbacks=[checkpoint, csv_logger])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")

# Evaluate model performance
print(classification_report(y_test, y_pred_classes))
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC-AUC: {roc_auc}")

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="blue", label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic Curve")
plt.legend(loc="lower right")
plt.show()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 204s 336ms/step - accuracy: 0.9667 - loss: 0.5116 - val_accuracy: 1.0000 - val_loss: 0.0073
Epoch 2/20
425/600 ━━━━━━━━━━━━━━━━━━━━ 56s 324ms/step - accuracy: 0.9940 - loss: 0.0729

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Predictions
predictions = model.predict(x_test).flatten()
print("\n🔹 First 10 Predictions vs Actual Values 🔹")
for i in range(10):
    print(f"Sample {i+1}: Actual = {y_test[i]}, Predicted Probability = {predictions[i]:.4f}")

# Save plots
os.makedirs(save_path, exist_ok=True)

# Histogram of predictions
plt.figure(figsize=(14, 10))
plt.hist(predictions, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel("Predicted Probability")
plt.ylabel("Count")
plt.title("Distribution of Predicted Probabilities")
plt.savefig(save_path + "histogram_predictions.png")
plt.show()

# Calculate the ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig(save_path + "750_new_if_roc_curve.png")

print(f"Area Under the Curve (AUC): {roc_auc:.4f}")

# Classification report
report = classification_report(y_test, (predictions > 0.5).astype(int))
print("\nClassification Report:" + report)
report = classification_report(y_test, (predictions > 0.5).astype(int), output_dict=True)

# Convert the dictionary into a DataFrame and transpose it
report_df = pd.DataFrame(report).transpose()

# Save the report to a CSV file
report_df.to_csv("750_new_if_classification_report.csv")

# Plot learning curves
plot_learning_curve(rnn_fit, save_path="750_new_if_learning_curve")


In [ ]:
import shap
explainer = shap.Explainer(model, input_data)

# Calculate SHAP values for the input data
shap_values = explainer(input_data)






In [ ]:
shap.summary_plot(shap_values, input_data)

In [ ]:
shap.initjs()  # Initialize JavaScript visualization
shap.force_plot(shap_values[0], input_data[0]

In [ ]:
shap.dependence_plot("feature_name", shap_values, input_data)

In [ ]:
shap.waterfall_plot(shap_values[0])